# What is PyTorch?

このノートブックでは，近年において最も主流な深層学習フレームワークのひとつであるPyTorchについて基本的な事項を説明し，以降で様々なモデルを実装していくための基礎を獲得することを目的とします．
# 目次

0.   準備
1.   テンソル生成方法
2.   種々の演算
3.   GPUの利用方法







## 準備
**ファイルの設定**　本稿は Google Colabolatory 上で実行されることを想定して作成されています．
以降でGPUを用いた演算を解説する際の準備として，はじめにファイルの設定を変更します．

本稿を Google Colabolatory 上で開いたのち，上部メニューから「編集」→「ノートブックの設定」を選択し，ハードウェアアクセラレータをGPUに設定してください．この設定変更はそれまでに実行されていたランタイムを初期化するため，必ず最初に行うようにしてください．

以降，GPUによる演算が必要なノートブックでは適宜同様の設定を行なってください．

**PyTorchのインポート** Google Colabolatory ではPyTorchがあらかじめインストールされています．

In [1]:
import torch                   # PyTorchのインポート
print(torch.__version__)       # バージョン確認
print(torch.cuda.is_available()) # GPUが正しく利用できるかの確認
import numpy as np # numpyのインポート

1.11.0+cu113
True


## テンソルの生成方法
----
**サイズを指定したテンソル生成**

任意サイズのテンソルを生成するために，単純に領域を確保することができます．値の初期化はなされません．

以下では例としてベクトル・行列・高階テンソルの領域をそれぞれ確保しています．同様に，任意の自然数を列挙することで様々なサイズのテンソルを生成することができます．

In [68]:
# 1D vector
x = torch.empty(3)
print(x)

tensor([-5.1600e-09,  0.0000e+00, -2.0000e+00])


In [69]:
# 2D matrix
x = torch.empty(2, 3)
print(x)

tensor([[-5.1600e-09,  0.0000e+00,  2.0000e+00],
        [ 0.0000e+00,         nan,  1.3438e-39]])


In [70]:
# 3D tensor
x = torch.empty(2, 3, 2)
print(x)

tensor([[[-5.1600e-09,  0.0000e+00],
         [ 2.0000e+00,  0.0000e+00],
         [        nan,  1.3438e-39]],

        [[-0.0000e+00, -1.6898e+00],
         [ 7.1853e+22,  3.0331e+32],
         [ 3.9415e+12,  7.5338e+28]]])


生成したテンソルのサイズは以下のように確認することができます．

In [71]:
print(x.size())  # テンソルのサイズを取得
print(x.size(0)) # テンソルの0次元目のサイズを取得

torch.Size([2, 3, 2])
2


また，0や1などの特定の値や任意の値で埋めたテンソルを生成することができます．

In [72]:
# 0で埋める
x = torch.zeros(2, 2)
print(x)

tensor([[0., 0.],
        [0., 0.]])


In [73]:
# 1で埋める
x = torch.ones(2, 2)
print(x)

tensor([[1., 1.],
        [1., 1.]])


In [74]:
# 任意の値（ここでは3.1415）で埋める
x = torch.full((2, 2), 3.1415)
print(x)

tensor([[3.1415, 3.1415],
        [3.1415, 3.1415]])


乱数によって初期化されたテンソルを生成することもできます．

In [75]:
# 標準正規分布で初期化
x = torch.randn(2, 2)
print(x)

tensor([[-0.0269, -0.7509],
        [-1.7603, -0.6986]])


In [76]:
# 0~1の一様分布で初期化
x = torch.rand(2, 2)
print(x)

tensor([[0.2784, 0.5126],
        [0.0915, 0.6979]])


----
**リストやnumpy.ndarrayからの変換**

あらかじめ定めた要素値によりテンソルを生成したい場合は，次のようにして実現できます．

In [77]:
# リストから生成
x = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
print(x)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


また，NumPy上でなんらかの計算を施した既存のndarrayをPyTorch上で利用したい場合も，同様のメソッドを用いることが可能です．このとき，ndarrayに設定されていた数値型がそのまま引き継がれます．

なお，print文による表示桁数は丸められてしまいますが，各要素には確かに数値型に対応する精度の数値が格納されています．

In [78]:
# 既存のndarrayとして準備
a = np.random.randn(3, 2)
print(a)
print(a.dtype)

[[-1.03311154 -1.44138005]
 [ 0.44631424  0.98689436]
 [-1.33760209 -1.10773672]]
float64


In [79]:
# numpy.ndarrayから生成
x = torch.tensor(a)
print(x)

tensor([[-1.0331, -1.4414],
        [ 0.4463,  0.9869],
        [-1.3376, -1.1077]], dtype=torch.float64)


----
**要素の取り出し**
リストやNumPyのndarrayと同様，スライスにより要素を取り出すことができます．

In [80]:
# 要素の取り出し
print(x[:2, :])
print(x[0, [1, 0]])

tensor([[-1.0331, -1.4414],
        [ 0.4463,  0.9869]], dtype=torch.float64)
tensor([-1.4414, -1.0331], dtype=torch.float64)


In [81]:
# 1要素のテンソルを単なるスカラー値に変換するために item を用いることができる
print(x[0, 0].item())

-1.033111538177395


----
**数値型**

PyTorchにもNumPyと同様に数値型の概念が存在します．数値型はテンソル生成時に指定できるほか，一度生成したテンソルを別の型に変換することもできます．

指定しない場合は基本的にfloat型とみなされます．よく用いられる型はfloat型およびlong型です．

In [82]:
# dtypeを指定することによるfloat型での生成（numpy.float32に対応）
x = torch.ones(2, 2, dtype=torch.float)
print(x)

tensor([[1., 1.],
        [1., 1.]])


既存のリストやnumpy.ndarrayを明示的にtorch.float型のテンソルへ変換することもできます．

In [83]:
# 先ほど用意した既存のndarray
print(a)
print(a.dtype)

[[-1.03311154 -1.44138005]
 [ 0.44631424  0.98689436]
 [-1.33760209 -1.10773672]]
float64


In [84]:
# numpy.ndarrayからtorch.floatのテンソルを生成
x = torch.FloatTensor(a)
print(x)
print(x.dtype)

tensor([[-1.0331, -1.4414],
        [ 0.4463,  0.9869],
        [-1.3376, -1.1077]])
torch.float32


In [85]:
# dtypeを指定することによるlong型での生成（numpy.int64に対応）
x = torch.ones(2, 2, dtype=torch.long)
print(x)

tensor([[1, 1],
        [1, 1]])


同様に，既存のリストやnumpy.ndarrayを明示的にtorch.long型のテンソルへ変換することができます．

In [86]:
# numpy.ndarrayからtorch.longのテンソルを生成
x = torch.LongTensor(a)
print(x)
print(x.dtype)

tensor([[-1, -1],
        [ 0,  0],
        [-1, -1]])
torch.int64


In [87]:
# torch.floatからtorch.longへの変換
f = torch.randn(4, 4)
print(f)

tensor([[ 0.5596, -0.7350,  1.2670,  1.1006],
        [ 1.9824, -0.1368, -0.2889,  0.2923],
        [ 0.2304,  0.1825,  0.3144, -0.3632],
        [ 0.3019,  1.2646, -1.0187, -0.6880]])


In [88]:
l = f.long()
print(l)

tensor([[ 0,  0,  1,  1],
        [ 1,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  1, -1,  0]])


In [89]:
# torch.longからtorch.floatへの変換
f2 = l.float()
print(f2)

tensor([[ 0.,  0.,  1.,  1.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.]])


---
**既存のテンソルと同サイズのテンソル生成**

様々なアルゴリズムを実装していくにあたり，同サイズのテンソルを複数用意して演算を行うという場面は頻出します．そこでPyTorchには，あるテンソルを基準に同サイズのテンソルを簡単に生成できるようにするための機能が備わっています．

これにより，全てのテンソルに同じサイズを明示的に割り当てるコードを書く冗長性の排除や，テンソルサイズを可変にしてもコードの見通しが良くなるといった利点が生まれます．

In [90]:
# 既存テンソル
x = torch.empty(2, 5, 2)
print(x.size())

torch.Size([2, 5, 2])


In [91]:
# xと同サイズの0埋めテンソルを生成
y = torch.zeros_like(x)
print(y)

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [92]:
# xと同サイズの1埋めテンソルを生成
y = torch.ones_like(x)
print(y)

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])


In [93]:
# xと同サイズで任意の値で埋めたテンソルを生成
y = torch.full_like(x, 3.1415)
print(y)

tensor([[[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]],

        [[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]]])


In [94]:
# xと同サイズで標準正規分布で初期化されたテンソルを生成
y = torch.randn_like(x)
print(y)

tensor([[[ 0.8659, -0.8685],
         [-2.1975, -0.2504],
         [-0.2125,  0.1848],
         [-0.0398, -1.3826],
         [-0.8886,  0.2944]],

        [[-1.3337,  0.1767],
         [-0.8329, -1.1323],
         [ 0.1931,  0.0686],
         [ 0.1744,  1.7541],
         [-1.2942,  0.4791]]])


In [95]:
# xと同サイズで0~1の一様分布で初期化されたテンソルを生成
y = torch.rand_like(x)
print(y)

tensor([[[0.0091, 0.7088],
         [0.1764, 0.0137],
         [0.0876, 0.5085],
         [0.8038, 0.7232],
         [0.3495, 0.5492]],

        [[0.0963, 0.8706],
         [0.8196, 0.8778],
         [0.1111, 0.2834],
         [0.3904, 0.8192],
         [0.9226, 0.1578]]])


## 種々の演算
----
**四則演算**　基礎的な四則演算をテンソル同士で行うことができます．

In [96]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[-0.9351, -0.1333]])
tensor([[-1.3806, -1.4930]])


In [97]:
# 加算
z = x + y
print(z)

tensor([[-2.3157, -1.6264]])


In [98]:
# 減算
z = x - y
print(z)

tensor([[0.4456, 1.3597]])


In [99]:
# 要素ごとの乗算
z = x * y
print(z)

tensor([[1.2910, 0.1991]])


In [100]:
# 要素ごとの除算
z = x / y
print(z)

tensor([[0.6773, 0.0893]])


In [101]:
# 冪乗
z = x**2
print(z)

tensor([[0.8743, 0.0178]])


---
**利用可能な数学関数** 様々な数学関数や集計関数が利用できます

In [102]:
x = torch.randn(1,2)
print(x)

tensor([[-1.0948, -1.3163]])


In [103]:
# 要素ごとの絶対値
z = torch.abs(x)
print(z)

tensor([[1.0948, 1.3163]])


In [104]:
# 要素ごとに三角関数を適用
z = torch.sin(x)
print(z)

tensor([[-0.8888, -0.9678]])


In [105]:
# 要素ごとに指数関数を適用
z = torch.exp(x)
print(z)

# 要素ごとに対数関数を適用
z = torch.log(x)
print(x)

tensor([[0.3346, 0.2681]])
tensor([[-1.0948, -1.3163]])


In [106]:
# 合計
z = torch.sum(x)
print(z)
# 最大値
z = torch.max(x)
print(z)
# 平均
z = torch.mean(x)
print(z)

tensor(-2.4111)
tensor(-1.0948)
tensor(-1.2055)


----
**テンソルサイズの変更**　
PyTorchのテンソルサイズはviewを用いて任意に整形できます．

In [107]:
x = torch.randn(2, 3, 4)
print(x)
print(x.size())

tensor([[[ 0.0925,  0.9412,  0.7084, -1.2347],
         [ 1.0614,  0.1593,  1.4961,  1.3085],
         [ 1.9321, -0.8235,  0.4778, -0.4326]],

        [[ 0.4504, -1.2953,  0.1997, -1.6016],
         [ 1.1109, -0.4056,  0.1390, -1.0322],
         [ 2.4118, -0.3277, -0.4056,  1.7963]]])
torch.Size([2, 3, 4])


In [108]:
y = x.view(6, 4)
print(y)
print(y.size())

tensor([[ 0.0925,  0.9412,  0.7084, -1.2347],
        [ 1.0614,  0.1593,  1.4961,  1.3085],
        [ 1.9321, -0.8235,  0.4778, -0.4326],
        [ 0.4504, -1.2953,  0.1997, -1.6016],
        [ 1.1109, -0.4056,  0.1390, -1.0322],
        [ 2.4118, -0.3277, -0.4056,  1.7963]])
torch.Size([6, 4])


In [109]:
# -1を指定すると整合するように自動で整数を推定してくれる
y = x.view(1, 2, -1, 2, 2)
print(y)
print(y.size())

tensor([[[[[ 0.0925,  0.9412],
           [ 0.7084, -1.2347]],

          [[ 1.0614,  0.1593],
           [ 1.4961,  1.3085]],

          [[ 1.9321, -0.8235],
           [ 0.4778, -0.4326]]],


         [[[ 0.4504, -1.2953],
           [ 0.1997, -1.6016]],

          [[ 1.1109, -0.4056],
           [ 0.1390, -1.0322]],

          [[ 2.4118, -0.3277],
           [-0.4056,  1.7963]]]]])
torch.Size([1, 2, 3, 2, 2])


**軸の入れ替え**　テンソルにおける任意の2軸の順番を入れ替えることができます．

In [110]:
y = x.transpose(2, 0)
print(y)
print(y.size())

tensor([[[ 0.0925,  0.4504],
         [ 1.0614,  1.1109],
         [ 1.9321,  2.4118]],

        [[ 0.9412, -1.2953],
         [ 0.1593, -0.4056],
         [-0.8235, -0.3277]],

        [[ 0.7084,  0.1997],
         [ 1.4961,  0.1390],
         [ 0.4778, -0.4056]],

        [[-1.2347, -1.6016],
         [ 1.3085, -1.0322],
         [-0.4326,  1.7963]]])
torch.Size([4, 3, 2])


2次元行列の場合は単純に転置することも可能です．

In [111]:
m = torch.randn(3, 2)
print(m)
print(m.size())

tensor([[-0.8680,  2.0168],
        [-0.4309,  0.3658],
        [-0.4620, -0.9455]])
torch.Size([3, 2])


In [112]:
# 転置行列を返す
n = m.t()
print(n)
print(n.size())

tensor([[-0.8680, -0.4309, -0.4620],
        [ 2.0168,  0.3658, -0.9455]])
torch.Size([2, 3])


全ての軸の順番を指定して入れ替えることも可能です．

In [113]:
# 軸の順列を指定して入れ替える
y = x.permute(2, 0, 1)
print(y)
print(y.size())

tensor([[[ 0.0925,  1.0614,  1.9321],
         [ 0.4504,  1.1109,  2.4118]],

        [[ 0.9412,  0.1593, -0.8235],
         [-1.2953, -0.4056, -0.3277]],

        [[ 0.7084,  1.4961,  0.4778],
         [ 0.1997,  0.1390, -0.4056]],

        [[-1.2347,  1.3085, -0.4326],
         [-1.6016, -1.0322,  1.7963]]])
torch.Size([4, 2, 3])


**メモリの整列**　上記にあげたテンソルの整形や軸の入れ替えは任意の操作数重ね合わせることが可能です．しかしながら，軸を入れ替えると表記上のテンソルと内部で確保されているメモリの並びが不整合となる場合があり，その後にviewメソッドを用いると失敗します．

In [114]:
# view size is not compatible with input tensor's size and stride... というエラーが生じる
# 今回はこの stride の部分に問題がある
z = x.transpose(2, 0).view(2, 2, 6)

RuntimeError: ignored

このようなエラーが発生した場合は，一度メモリの並びを整列させる操作を噛ませることで解消可能です．

In [136]:
z = x.transpose(2, 0).contiguous().view(2, 2, 6)
print(z)

tensor([[[ 0.0925,  0.4504,  1.0614,  1.1109,  1.9321,  2.4118],
         [ 0.9412, -1.2953,  0.1593, -0.4056, -0.8235, -0.3277]],

        [[ 0.7084,  0.1997,  1.4961,  0.1390,  0.4778, -0.4056],
         [-1.2347, -1.6016,  1.3085, -1.0322, -0.4326,  1.7963]]])


----
**in-placeな演算**　
通常の演算では，出力テンソルとして新たなメモリ領域が確保されています．しかしながら，メモリの使用量を抑制したい場合等に，入力テンソルの要素をそのまま置換するように演算を行うことができます．

*   加算：add_
*   減算：sub_
*   乗算：mul_
*   除算：div_
*   転置：t_
*   軸の入れ替え：transpose_
*   値のコピー：copy_

等が用意されています．



In [137]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[-1.2996,  1.1480]])
tensor([[1.9247, 0.3035]])


In [138]:
# xにyをin-placeに加算する
x.add_(y)
# x自体の値が演算結果で書き換わる
print(x)

tensor([[0.6251, 1.4515]])


In [139]:
# xをin-placeに転置する
x.t_()
print(x)

tensor([[0.6251],
        [1.4515]])


In [140]:
# xの軸をin-placeに入れ替える
x.transpose_(0,1)
print(x)

tensor([[0.6251, 1.4515]])


In [141]:
# xにyの値をコピーする
x.copy_(y)
print(x)

tensor([[1.9247, 0.3035]])


ただし，後続のノートブックで説明する自動微分機能を用いる際は，基本的にin-placeな演算を行うことはできないことに留意してください．

## GPUの利用方法
----
**生成済のテンソルをGPUメモリに載せる**ことができます．複数枚のGPUが搭載されたサーバ上で実行する場合はGPU番号を指定することが推奨されますが，Google Colaboratory は1枚のみの搭載のため以下のようにして簡単に既存テンソルをGPU上のテンソル（CUDA Tensor）に変換することができます．

In [142]:
x_cuda = x.cuda()
print(x_cuda)

tensor([[1.9247, 0.3035]], device='cuda:0')


テンソルの生成時にdeviceを指定することで，**初めからGPU上のメモリを確保してテンソルを生成する**ことも可能です．

In [143]:
x = torch.randn(1, 2, device="cuda:0")
print(x)

tensor([[-0.0947, -1.3762]], device='cuda:0')


存在しないGPU番号を指定するとエラーが発生します．

In [144]:
x = torch.randn(1, 2, device="cuda:1")

RuntimeError: ignored

同じデバイス上のテンソルを用いて，先に紹介したような演算を同様に行うことが可能です．
ただし，異なるデバイス同士（CPUとGPU，または異なる番号のGPU同士）での演算はできません．

**CPUへの変換**　GPU上のテンソルは，同様に簡単にCPU上に移行することが可能です．

In [145]:
# CPUへ
x_cpu = x_cuda.cpu()
print(x_cpu)
print(x_cpu.device)

tensor([[1.9247, 0.3035]])
cpu


**統一的な記法**　PyTorchが扱うテンソルは，デバイス名を文字列で指定することで簡単にデバイス間を移動させることができます．

In [146]:
# CPU上で生成
x = torch.randn(2, 2)
print(x.device)

cpu


In [147]:
# 0番GPUへ
x_cuda0 = x.to("cuda:0")
print(x_cuda0.device)

cuda:0


In [148]:
# CPUへ
x_cpu = x_cuda0.to("cpu")
print(x_cpu.device)

cpu


さらに，既存のテンソルを引数として，同じデバイス・同じ数値型への変換を行うことも可能です．

In [149]:
# 0番GPUにdouble型のテンソルを生成
d_cuda0 = torch.randn(2, 2, dtype=torch.double, device="cuda:0")
print(d_cuda0.device)
print(d_cuda0.dtype)

cuda:0
torch.float64


In [150]:
# xをd_cuda0と同じデバイス・数値型へ変換する
x_d = x.to(d_cuda0)
print(x_d.device)
print(x_d.dtype)

cuda:0
torch.float64


**他の配列オブジェクトへの変換**　CPU上のテンソルに限り，numpy.ndarrayやリストなどへ変換することができます．

In [151]:
# numpy.ndarrayに変換
x_ndarray = x_cpu.numpy()
print(x_ndarray)

[[2.5855193  0.70649797]
 [0.63914067 0.3745526 ]]


In [152]:
# リストに変換
x_list = x_cpu.tolist()
print(x_list)

[[2.585519313812256, 0.7064979672431946], [0.6391406655311584, 0.3745526075363159]]


In [153]:
x = np.random.randn(3,2)
y = np.random.randn(3,2)
print(x)
print(y)
z=(x - y)**2
print(z)
s = np.sum(z, axis=1)
print(s)
m = np.mean(s)
print(m)

[[ 0.12027179  0.72961023]
 [ 0.16246096  1.58056949]
 [-0.61118028  0.07540544]]
[[ 0.75738417 -0.27705104]
 [ 1.48525933 -2.47775135]
 [ 0.39230461  0.31453851]]
[[ 0.40591218  1.01336692]
 [ 1.74979553 16.46996805]
 [ 1.00698191  0.05718463]]
[ 1.4192791  18.21976358  1.06416654]
6.901069737663568


# 課題
<h4>
平均二乗誤差関数の実装
</h4>

 
入力：$\boldsymbol{X}=(\boldsymbol{x}_1,\boldsymbol{x}_2,\cdots\boldsymbol{x}_{N})^{T}\in\mathbb{R}^{N\times M},　\boldsymbol{Y}=(\boldsymbol{y}_1,\boldsymbol{y}_2,\cdots\boldsymbol{y}_{N})^{T}\in\mathbb{R}^{N\times M}$
      
出力：$z\in\mathbb{R}, z=L(\boldsymbol{X}, \boldsymbol{Y}~)$
    
    

- 平均二乗誤差 :
$$ 
L(\boldsymbol{X},\boldsymbol{Y}~) = \frac{1}{N}\sum^N_i \|\boldsymbol{x}_i-\boldsymbol{y}_i\|_2^2\\
$$
Numpyのndarrayを受け取り，GPU上で平均二乗誤差を計算して，GPU上のメモリに乗ったテンソルを出力する関数を実装してみましょう


In [2]:
def MSE(x, y):
  x = torch.tensor(x).cuda()
  y = torch.tensor(y).cuda()
  z = torch.mean(torch.sum((x - y) ** 2, dim=1))
  return z

In [3]:
from google.colab import drive # driveを接続
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# drive中の課題ファイルのあるディレクトリに移動
%cd /content/drive/My Drive/Colab Notebooks/先端人工知能論I/20220607/data/

from test import *

/content/drive/My Drive/Colab Notebooks/先端人工知能論I/20220607/data


In [5]:
test_MSE(MSE)

OK


OKと出力されれば完成です

最終項で紹介したGPU上での演算は極めて重要であり，今日では専らGPUを用いて演算することが主流となっています．
というのも，一般の配列操作や行列演算等はGPUを用いることで高速に並列計算可能なためです．このような装置の恩恵もあって今日の深層学習モデルは現実的な時間で学習させることが可能となっています．

----
お疲れ様でした．本稿は以上で終了いたします．
以降のハンズオンでは、深層学習において重要な概念のひとつである自動微分機能について解説したのち，具体的なデータセットとタスクを通してより深層学習モデルに関する知見を深めていきます．